# SEACells Versions

### Imports

In [1]:
import tracemalloc
import time
import scanpy as sc
import pandas as pd

In [2]:
import cupy as cp
import cupyx
import numpy as np
from tqdm import tqdm

from icecream import ic

In [3]:
# from importlib import reload
from SEACells.core import SEACells
# reload(SEACells)

findfont: Font family ['Raleway'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Lato'] not found. Falling back to DejaVu Sans.


### Initialization

In [4]:
num_cells = 1000

In [5]:
ad = sc.read("/home/aparna/DATA/aparnakumar/50000_cells/mouse_marioni_50k.h5ad") 
num_cells = 1000
ad = ad[:num_cells]

In [6]:
def get_data(ad, num_cells, use_gpu, use_sparse): 
  ## User defined parameters

  ## Core parameters 
  # number of SEACells
  n_SEACells = num_cells // 75
  build_kernel_on = 'X_pca' # key in ad.obsm to use for computing metacells
                            # This would be replaced by 'X_svd' for ATAC data

  ## Additional parameters
  n_waypoint_eigs = 10 # Number of eigenvalues to consider when initializing metacells

  model = SEACells(ad, 
                                 use_gpu=use_gpu, 
                                 use_sparse=use_sparse, 
                                 build_kernel_on=build_kernel_on, 
                                 n_SEACells=n_SEACells, 
                                 n_waypoint_eigs=n_waypoint_eigs,
                                 convergence_epsilon = 1e-5)
  model.construct_kernel_matrix()
  model.initialize_archetypes()

  start = time.time()
  tracemalloc.start()

  model.fit(min_iter=10, max_iter=150)

  end = time.time()
  tot_time = end - start

  mem = tracemalloc.get_traced_memory()
  tracemalloc.stop()

  assignments = model.get_hard_assignments()
  
  return assignments, tot_time, mem

In [7]:
def all_versions(ad):
    assignments1, time1, mem1 = get_data(ad, num_cells = num_cells, use_gpu = False, use_sparse=False)
    assignments2, time2, mem2 = get_data(ad, num_cells = num_cells, use_gpu = False, use_sparse= True)
    assignments3, time3, mem3 = get_data(ad, num_cells = num_cells, use_gpu = True, use_sparse=False)
    assignments4, time4, mem4 = get_data(ad, num_cells = num_cells, use_gpu = True, use_sparse= True)

    # Write the assignments
    assignments = [assignments1, assignments2, assignments3, assignments4] 
    
    # Write the time and memory data
    comparisons = pd.DataFrame({'version': ['v1: no GPU, no sparse', 'v2:  no GPU, yes sparse', 'v3: yes GPU, no sparse', 'v4: yes GPU, yes sparse'], 
                           'time (s)': [time1, time2, time3, time4],
                           'peak memory': [mem1[1], mem2[1], mem3[1], mem4[1]]})
    
    return assignments, comparisons

In [8]:
B3 = cp.random.random((13, 1000))
B3 = cp.sparse.csr_matrix(B3)
B3 =B3.argmin(axis=0)

In [9]:
import scipy.sparse

ATA = cp.random.random((1000, 1000))
ATA = cp.sparse.csr_matrix(ATA)


f = ((ATA.multiply(ATA)).sum(axis=0)).ravel()
g = ATA.diagonal().ravel()

k =  13
n = 1000
d = cp.sparse.csr_matrix((k, n), dtype=cp.float64)
omega = cp.sparse.csr_matrix((k, n), dtype=cp.float64)

# keep track of selected indices
centers = cp.sparse.csr_matrix((k, n), dtype=cp.float64)

# sampling
for j in tqdm(range(k)):
        score = f / g
        p = score.argmax()
        
        delta_term1 = ATA[:, p].toarray().squeeze()
        #     # print(delta_term1)
        ic(omega[:, p].shape)
        ic(type(omega[:, p].reshape(-1, 1).multiply(omega)))
        delta_term2 = omega[:, p].reshape(-1, 1).multiply(omega).sum(axis=0).squeeze()
        delta = delta_term1 - delta_term2

        #     # some weird rounding errors
        # BOOKMARK
        #delta[p] = [0, delta[p]].max()

        #     o = delta / np.max([np.sqrt(delta[p]), 1e-6])
        #     omega_square_norm = np.linalg.norm(o) ** 2
        #     omega_hadamard = np.multiply(o, o)
        #     term1 = omega_square_norm * omega_hadamard

        #     # update f (term2)
        #     pl = np.zeros(n)
        #     for r in range(j):
        #         omega_r = omega[r, :]
        #         pl += np.dot(omega_r, o) * omega_r

        #     ATAo = (ATA @ o.reshape(-1, 1)).ravel()
        #     term2 = np.multiply(o, ATAo - pl)

        #     # update f
        #     f += -2.0 * term2 + term1

        #     # update g
        #     g += omega_hadamard

        #     # store omega and delta
        #     d[j, :] = delta
        #     omega[j, :] = o

        #     # add index
        #     centers[j] = int(p)

        # return centers

  0%|          | 0/13 [00:00<?, ?it/s]

ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
  8%|▊         | 1/13 [00:00<00:02,  5.73it/s]ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
 31%|███       | 4/13 [00:00<00:01,  7.48it/s]ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'

In [10]:
delta_term1.shape

(1000,)

In [11]:
p = cp.array([121])
omega = cp.sparse.csr_matrix((14, 1000), dtype=cp.float64)
thing = omega[:, p].reshape(-1, 1).multiply(omega).sum(axis=0).squeeze()
ic(thing.shape)
ic(type(thing))


type(thing.sum(axis=0))



ic| thing.shape: (1000,)
ic| type(thing): <class 'cupy.ndarray'>


cupy.ndarray

In [12]:
n = 1000
k = 14

# precompute M.T * M
# ATA = M.T @ M
ATA = cupyx.scipy.sparse.csr_matrix((n, n))


f = cp.array((ATA.multiply(ATA)).sum(axis=0)).ravel()
g = cp.array(ATA.diagonal()).ravel()

d = cp.sparse.csr_matrix((k, n))
omega = cp.sparse.csr_matrix((k, n))

# keep track of selected indices
centers = cp.sparse.csr_matrix((k, n))

# sampling
for j in tqdm(range(k)):
    # Compute score, dividing the sparse f by the sparse g
    score = f / g
            
    # Compute p, which is the largest score
    p = cp.argmax(score)

    # Compute delta_term1 to be the column of ATA at index p
    delta_term1 = ATA[:, p].toarray().squeeze()

    # Compute delta_term2 to be the sum of the outer product of omega and itself
    delta_term2 = (omega[:, p].reshape(-1, 1).multiply(omega).sum(axis=0).squeeze())
    delta = delta_term1 - delta_term2

    # some weird rounding errors
    delta[p] = max([0, delta[p]])

    o = delta / max([cp.sqrt(delta[p]), 1e-6])
    omega_square_norm = cp.linalg.norm(o) ** 2
    omega_hadamard = cp.multiply(o, o)
    term1 = omega_square_norm * omega_hadamard

    # update f (term2)
    pl = cp.zeros(n)
    for r in range(j):
        omega_r = omega[r, :]
        pl += omega_r.dot(o) * omega_r

    ATAo = (ATA @ o.reshape(-1, 1)).ravel()
    term2 = o *  (ATAo - pl)

    # update f
    f += -2.0 * term2 + term1

    # update g
    g += omega_hadamard

    # store omega and delta
    d[j, :] = delta
    omega[j, :] = o

    # add index
    centers[j] = int(p)

centers

  0%|          | 0/14 [00:00<?, ?it/s]/home/aparna/miniconda/envs/env_SEACells/lib/python3.8/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '
100%|██████████| 14/14 [00:00<00:00, 71.63it/s]


In [13]:
import cupy

kernel_matrix =  cupyx.scipy.sparse.rand(1000, 1000, density=0.01, format='csr', dtype=np.float64)
# Make a sparse csr matrix of random data 
reconstruction = cupyx.scipy.sparse.rand(1000, 1000, density=0.01, format='csr', dtype=np.float64)

ic(type(kernel_matrix))
ic(type(reconstruction))



cp.linalg.norm(kernel_matrix - reconstruction)

ic| type(kernel_matrix): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| type(reconstruction): <class 'cupyx.scipy.sparse._csr.csr_matrix'>


array(nan)

In [14]:
assignments4, time4, mem4 = get_data(ad, num_cells = num_cells, use_gpu = True, use_sparse= True)

SPARSE AND GPU
TRYING SEACellsGPU
Welcome to SEACells GPU!
build_graph.SEACellGraph completed
Computing kNN graph using scanpy NN ...
Computing radius for adaptive bandwidth kernel...



Making graph symmetric...
Parameter graph_construction = union being used to build KNN graph...
Computing RBF kernel...



Building similarity LIL matrix...


  0%|          | 0/14 [00:00<?, ?it/s]


Constructing CSR matrix...
Building kernel on X_pca
Computing diffusion components from X_pca for waypoint initialization ... 
Determing nearest neighbor graph...
Done.
Sampling waypoints ...
Done.
Selecting 9 cells from waypoint initialization.
Initializing residual matrix using greedy column selection
Initializing f and g...


/home/aparna/miniconda/envs/env_SEACells/lib/python3.8/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '
100%|██████████| 14/14 [00:00<00:00, 75.36it/s]


Selecting 4 cells from greedy initialization.
Randomly initialized A matrix.
Convergence threshold set to 0.000771444373885075 based on epsilon = 1e-05
Starting iteration 1.
Completed iteration 1.


/home/aparna/miniconda/envs/env_SEACells/lib/python3.8/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '


Starting iteration 10.
Completed iteration 10.
Starting iteration 20.
Completed iteration 20.
Starting iteration 30.
Completed iteration 30.
Starting iteration 40.
Completed iteration 40.
Starting iteration 50.
Completed iteration 50.
Starting iteration 60.
Completed iteration 60.
Starting iteration 70.
Completed iteration 70.
Starting iteration 80.
Completed iteration 80.
Starting iteration 90.
Completed iteration 90.
Starting iteration 100.
Completed iteration 100.
Starting iteration 110.
Completed iteration 110.
Starting iteration 120.
Completed iteration 120.
Starting iteration 130.
Completed iteration 130.
Starting iteration 140.
Completed iteration 140.
Starting iteration 150.
Completed iteration 150.


RuntimeWarning: Warning: Algorithm has not converged - you may need to increase the maximum number of iterations

In [ ]:
assignments, comparisons = all_versions(ad)

NOT SPARSE AND NOT GPUT
TRYING SEACellsCPUDense
Welcome to SEACells!
Computing kNN graph using scanpy NN ...
Computing radius for adaptive bandwidth kernel...



Making graph symmetric...
Parameter graph_construction = union being used to build KNN graph...
Computing RBF kernel...



Building similarity LIL matrix...


100%|██████████| 14/14 [00:00<00:00, 1212.73it/s]


Constructing CSR matrix...
Building kernel on X_pca
Computing diffusion components from X_pca for waypoint initialization ... 
Determing nearest neighbor graph...
Done.
Sampling waypoints ...
Done.
Selecting 9 cells from waypoint initialization.
Initializing residual matrix using greedy column selection
Initializing f and g...
Selecting 4 cells from greedy initialization.
Randomly initialized A matrix.
Setting convergence threshold at 0.00056
Starting iteration 1.


...

Completed iteration 1.


ic| f.A_ = A: [array(66.13703115), array(74.74279219)]
ic| # ic(type(dif: <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| #ic(scipy.sparse.linalg.norm: 2196697088


COMPUTE RECONSTRUCTION
FINISHED COMPUTE RECONSTRUCTION


ic| def plot_convergence(self,: array(60.26921085)
ic| """Plot behaviour of sq: <1000x1000 sparse matrix of type '<class 'numpy.float64'>'
                             	with 1000000 stored elements in Compressed Sparse Row format>
ic| del A, B, RSS: [array(66.13703115), array(74.74279219), array(60.26921085)]
ic| = pd.DataFra: (13, 1000)
ic| df = pd.DataFrame({"SEACel: (1, 1000)
ic| df.index = self.ad.obs_names: (1000,)
ic| turn df
    
    def get_ha: (1000,)
ic| f.A_ = A: [array(66.13703115), array(74.74279219), array(60.26921085)]
ic| # ic(type(dif: <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| #ic(scipy.sparse.linalg.norm: 2196697088


COMPUTE RECONSTRUCTION
FINISHED COMPUTE RECONSTRUCTION
